### 📦 Imports

In [ ]:
import cv2
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from torchvision import models, transforms
import torch
from torch.nn.functional import adaptive_avg_pool2d
from matplotlib import pyplot as plt

### ⚙️ Configuration

In [ ]:
VIDEO_1_PATH = 'broadcast.mp4'
VIDEO_2_PATH = 'tacticam.mp4'
MODEL_PATH = 'best.pt'
OUTPUT_DIR = 'output_frames'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

os.makedirs(OUTPUT_DIR, exist_ok=True)

### 🧠 Load YOLOv5 Model

In [ ]:
model_yolo = torch.hub.load('ultralytics/yolov5', 'custom', path=MODEL_PATH, force_reload=True)
model_yolo.to(DEVICE)

### 🔍 Load ResNet18 Feature Extractor

In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

model_feat = models.resnet18(pretrained=True)
model_feat = model_feat.to(DEVICE)
model_feat.eval()

### 🧩 Feature & Detection Functions

In [ ]:
def extract_features(image):
    image = transform(image).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        features = model_feat(image)
    return features.view(-1).cpu().numpy()

def detect_players(frame):
    results = model_yolo(frame)
    detections = results.xyxy[0]
    players = []
    for *box, conf, cls in detections:
        if int(cls) == 0:
            x1, y1, x2, y2 = map(int, box)
            crop = frame[y1:y2, x1:x2]
            if crop.size > 0:
                players.append(crop)
    return players

### 🎥 Video Processing Function

In [ ]:
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    features_list = []
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret or frame_idx > 0:
            break
        players = detect_players(frame)
        for idx, player in enumerate(players):
            feat = extract_features(player)
            features_list.append((f"player_{idx}", feat))
        frame_idx += 1
    cap.release()
    return features_list

### 🔗 Player Matching Function

In [ ]:
def match_players(features1, features2):
    matched = {}
    for id2, feat2 in features2:
        similarities = [(id1, cosine_similarity([feat1], [feat2])[0][0]) for id1, feat1 in features1]
        best_match = max(similarities, key=lambda x: x[1])
        matched[id2] = best_match[0]
    return matched

### 🚀 Run Matching Pipeline

In [ ]:
print("Processing broadcast video...")
features_broadcast = process_video(VIDEO_1_PATH)

print("Processing tacticam video...")
features_tacticam = process_video(VIDEO_2_PATH)

print("Matching players...")
matches = match_players(features_broadcast, features_tacticam)

print("\nMatched Player IDs:")
for tacticam_id, broadcast_id in matches.items():
    print(f"{tacticam_id} -> {broadcast_id}")

with open("player_id_mapping.txt", "w") as f:
    for tacticam_id, broadcast_id in matches.items():
        f.write(f"{tacticam_id} -> {broadcast_id}\n")

print("\nMapping saved to player_id_mapping.txt")